In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## Shifting and rescaling data

In [14]:
#receives a csv and returns a regularized dataframe
def regularize_data(X):
    df = pd.read_csv(X)
    df_copy = df.copy()
    continuous_features = ['sbp', 'tobacco', 'ldl', 'adiposity','typea', 'obesity', 'alcohol', 'age']

    for feat in continuous_features:
        #shifts the values to have mean 0 and std 1
        df_copy[feat] = (df[feat] - df[feat].mean()) / df[feat].std()

    #normalize non continuous features
    df_copy['famhist'] = df_copy['famhist'].map({'Present': 1, 'Absent': 0})

    return df_copy

regularize_data('data.csv')

,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,1,1.057417,1.821099,0.477894,-0.295183,1,-0.418017,-0.176594,3.274189,0.628654,1
1,2,0.276789,-0.789382,-0.159507,0.411694,0,0.193134,0.670646,-0.612081,1.381617,1
2,3,-0.991731,-0.774141,-0.608585,0.883374,1,-0.112441,0.734723,-0.540597,0.217947,0
3,4,1.545310,0.841352,0.806252,1.622382,1,-0.214300,1.411091,0.294742,1.039361,1
4,5,-0.211103,2.169453,-0.598928,0.305020,1,0.702427,-0.012842,1.645991,0.423301,1
...,...,...,...,...,...,...,...,...,...,...,...
457,459,3.692037,-0.704470,0.598614,0.811401,0,1.109862,0.570971,-0.696228,1.039361,0
458,460,2.130781,0.122871,-0.159507,0.860240,0,-0.112441,0.608942,0.068445,0.628654,1
459,461,-1.479624,-0.138395,-1.521228,-1.307946,0,-1.334744,-1.413043,0.391960,0.834008,0
460,462,-0.991731,0.384137,3.317227,0.691875,0,1.109862,0.309916,0.282897,-0.192760,0
